In [81]:
import pandas as pd
import graphviz
import uuid

class Gini():
    def __init__(self, file) -> None:
        self.df = pd.read_csv(file)
        self.df = pd.DataFrame(self.df)
        self.tree = [] # liste qui contient les seuils optimaux, avec leur gini et variable
        self.node_info = {}
        self.main()

    def division(self, df):
        # stocker les valeurs des indices de Gini, leurs seuils et variables correspondantes
        tab_gini = pd.DataFrame(columns=['variable', 'seuil', 'gini'])

        # calcul de Dk
        p1 = df['Y'].value_counts().A  # nombre de modalités A dans ce noeud
        p2 = df['Y'].value_counts().B
        n = p1 + p2  # nombre d'individus
        Dk = 1 - ((p1/n)**2 + (p2/n)**2)  # indice de Gini avant séparation

        # parcourir chaque variable X1, X2...
        i = 0
        for x in df.columns[1:]:
            # parcourir chaque valeur de cette variable comme seuil
            for seuil in df[x]:
                # liste des modalités du noeud de gauche respectant le seuil
                noeud_gauche = df[df[x] <= seuil].Y
                noeud_droite = df[df[x] > seuil].Y

                # calculer l'indice de gini seulement lorsque le noeud n'est pas totalement pur (contient des modalités différentes)
                Dkg, Dkd = 0, 0
                if 'A' in noeud_gauche.tolist() and 'B' in noeud_gauche.tolist():
                    p1g = noeud_gauche.value_counts().A
                    p2g = noeud_gauche.value_counts().B
                    ng = p1g + p2g
                    Dkg = (1 - ((p1g/ng)**2 + (p2g/ng)**2)) * \
                        (ng/n)  # indice de Gini de ce noeud

                if 'A' in noeud_droite.tolist() and 'B' in noeud_droite.tolist():
                    p1d = noeud_droite.value_counts().A
                    p2d = noeud_droite.value_counts().B
                    nd = p1d + p2d
                    Dkd = (1 - ((p1d/nd)**2 + (p2d/nd)**2)) * (nd/n)

                gini = Dk - (Dkg + Dkd)  # indice de Gini global
                tab_gini.loc[i] = [x, seuil, gini]  # ajout des informations
                i += 1

        tab_gini.sort_values(by="gini", ascending=False, inplace=True)
        tab_gini.reset_index(drop=True, inplace=True)
        return tab_gini.iloc[0]

    def recursion(self, df, level=0, parent_node_id=None):
        result = self.division(df)
        node_id = len(self.tree)
        self.tree.append([level, result['variable'], result['seuil'], result['gini'], node_id])
        
        # Ajouter les informations du noeud dans node_info
        modalites_gauche = list(df[df[result['variable']] <= result['seuil']].Y)
        modalites_droite = list(df[df[result['variable']] > result['seuil']].Y)
        self.node_info[node_id] = {'parent': parent_node_id, 'children': [], 'info_gauche': modalites_gauche, 'info_droite': modalites_droite}
        
        if parent_node_id is not None:
            self.node_info[parent_node_id]['children'].append(node_id)

        df_gauche = df[df[result['variable']] <= result['seuil']]
        df_droite = df[df[result['variable']] > result['seuil']]
        
        if 'A' in df_gauche.Y.tolist() and 'B' in df_gauche.Y.tolist():
            self.recursion(df_gauche, level + 1, node_id)
        
        if 'A' in df_droite.Y.tolist() and 'B' in df_droite.Y.tolist():
            self.recursion(df_droite, level + 1, node_id)
            

    def display(self):
        dot = graphviz.Digraph(comment='Arbre de décision', graph_attr={'size': '10,10!'})
        for noeud in self.tree:
            level, variable, seuil, gini, node_id = noeud
            modalites_gauche, modalites_droite = "", ""
            if level > 0:
                info_gauche = self.node_info[node_id]['info_gauche']
                info_droite = self.node_info[node_id]['info_droite']
                if 'A' in info_gauche and 'B' in info_gauche:
                    info_gauche = None
                if 'A' in info_droite and 'B' in info_droite:
                    info_droite = None
                if info_gauche:
                    gauche_id = str(uuid.uuid4())
                    dot.node(gauche_id, label=f"Modalités : {info_gauche}", shape="oval")
                    dot.edge(f"{node_id}", gauche_id, label=f"<= {seuil}")
                if info_droite:
                    droite_id = str(uuid.uuid4())
                    dot.node(droite_id, label=f"Modalités : {info_droite}", shape="oval")
                    dot.edge(f"{node_id}", droite_id, label=f"> {seuil}")
            label = f"Indice de Gini : {gini:.2f}{modalites_gauche}{modalites_droite}"
            if info_gauche is None and info_droite is None:
                continue
            dot.node(f"{node_id}", label=label, shape="oval")
            parent_id = self.node_info[node_id]['parent']
            parent_node = [n for n in self.tree if n[-1] == parent_id][0]
            parent_variable, parent_seuil, _, _, _ = parent_node
            signe = f"<= {parent_seuil}" if variable == parent_variable else f"> {parent_seuil}"
            dot.edge(f"{parent_id}", f"{node_id}", label=signe)
        dot.render("arbre_decision", view=True, format="png")





    def main(self):
        self.recursion(self.df)
        self.display() # afficher l'arbre avec graphviz
        print(self.tree)
        print(self.node_info)

if __name__ == "__main__":
    Gini('data.csv')


[[0, 'X1', 10.0, 0.1493827160493828, 0], [1, 'X2', 0.8, 0.375, 1], [1, 'X1', 12.0, 0.11999999999999983, 2], [2, 'X1', 13.0, 0.5, 3]]
{0: {'parent': None, 'children': [1, 2], 'info_gauche': ['A', 'B', 'B', 'B'], 'info_droite': ['A', 'A', 'A', 'A', 'B']}, 1: {'parent': 0, 'children': [], 'info_gauche': ['A'], 'info_droite': ['B', 'B', 'B']}, 2: {'parent': 0, 'children': [3], 'info_gauche': ['A', 'A', 'A'], 'info_droite': ['A', 'B']}, 3: {'parent': 2, 'children': [], 'info_gauche': ['B'], 'info_droite': ['A']}}
